## Dependencies

In [1]:
%%bash
mamba create -n EnveomicsLab6 -c conda-forge -c bioconda maxbin2 r-optparse r-fitdistrplus r-sn r-investr
mamba activate EnveomicsLab6


Looking for: ['maxbin2', 'r-optparse', 'r-fitdistrplus', 'r-sn', 'r-investr']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
bioconda/linux-64                                           Using cache
bioconda/noarch                                             Using cache
anaconda/linux-64                                           Using cache
anaconda/noarch                                             Using cache


warning  libmamba Extracted package cache '/home/k_nishiura/opt/miniconda3/pkgs/libgcc-7.2.0-h69d50b8_2' has invalid url


Transaction

  Prefix: /home/k_nishiura/opt/miniconda3/envs/EnveomicsLab6

  Updating specs:

   - maxbin2
   - r-optparse
   - r-fitdistrplus
   - r-sn
   - r-investr


  Package                           Version  Build                Channel                    Size
───────────────────────────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex                       0.1  conda_forge          conda-forge/linux-64        3kB
  + _openmp_mutex                       4.5  2_gnu                conda-forge/linux-64       24kB
  + _r-mutex                          1.0.1  anacondar_1          conda-forge/noarch          4kB
  + binutils_impl_linux-64           2.36.1  h193b22a_2           conda-forge/linux-64     Cached
  + bowtie2                           2.4.5  py310h8d7afc0_4      bioconda/linux-64          15MB
  + bwidget                   

## Step 01: Retrieve the data
Create directories and download data

In [3]:
%%bash

mkdir -p 00_Reads_QCed
wget http://enve-omics.ce.gatech.edu/data/GATech_6720/Lab05/T4AerOil_sbsmpl5.fa.gz
gunzip T4AerOil_sbsmpl5.fa.gz
mv T4AerOil_sbsmpl5.fa 00_Reads_QCed

--2022-10-03 23:59:02--  http://enve-omics.ce.gatech.edu/data/GATech_6720/Lab05/T4AerOil_sbsmpl5.fa.gz
Resolving enve-omics.ce.gatech.edu (enve-omics.ce.gatech.edu)... 128.61.130.232
Connecting to enve-omics.ce.gatech.edu (enve-omics.ce.gatech.edu)|128.61.130.232|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97962056 (93M) [application/x-gzip]
Saving to: ‘T4AerOil_sbsmpl5.fa.gz’

     0K .......... .......... .......... .......... ..........  0%  394K 4m3s
    50K .......... .......... .......... .......... ..........  0% 1.28M 2m38s
   100K .......... .......... .......... .......... ..........  0% 1.45M 2m6s
   150K .......... .......... .......... .......... ..........  0% 5.35M 99s
   200K .......... .......... .......... .......... ..........  0% 1.52M 92s
   250K .......... .......... .......... .......... ..........  0% 2.75M 82s
   300K .......... .......... .......... .......... ..........  0% 6.58M 72s
   350K .......... .......... .......... ......

## Step 02: Assemble the metagenome.
The file we just downloaded contains paired Illumina sequence reads in interleaved format. Interleaved simply means that the first read pair is always immediately followed by the second read pair on the next line. The next computational challenge here is to assemble reads together that represent the same species (metagenome assembly). We will use the IDBA-UD assembler for this task.

In [ ]:
%%bash

idba_ud -r 00_Reads_QCed/T4AerOil_sbsmpl5.fa  --min_contig 1000 -o 01_IDBA_Assembly

# alternative assemblers

# install
# mamba install -c bioconda spades -y
# run 
# spades.py --meta --12 00_Reads_QCed/T4AerOil_sbsmpl5.fa -o 01_SPAdes_Assembly

# install 
# mamba install -c bioconda megahit -y
# run
# megahit --12 00_Reads_QCed/T4AerOil_sbsmpl5.fa -o 01_Megahit_Assembly

## Step 03: Cluster the assembly into bins (MAGs)
With the current state of DNA sequencing technology, it is not typically possible to reconstruct complete genomes from the short Illumina sequenced reads based on assembly alone. What we end up with after the assembly process are hundreds or thousands of sections of contiguous sequences (contigs) representing the different species in the sample. The next computational challenge is to sort out which contigs belong to which genomes (species) and group them into population genome bins to obtain MAGs. We will use MaxBin2 for this task.

Further reading (alternative binning tools):
1.	MetaBat [publication](https://peerj.com/articles/7359/), [website](https://bitbucket.org/berkeleylab/metabat/src/master/)
2.	Concoct [publication](https://doi.org/10.1038/nmeth.3103), [website](https://github.com/BinPro/CONCOCT)
3.	BinSanity [publication](https://peerj.com/articles/3035/), [website](https://github.com/edgraham/BinSanity)
4.	DasTool [publication](https://doi.org/10.1038/s41564-018-0171-1), [website](https://github.com/cmks/DAS_Tool)
5.	CAMI challenge [publication](https://doi.org/10.1038/nmeth.4458), [website](https://data.cami-challenge.org/)

In [ ]:
%%bash

# shortcut:
wget http://enve-omics.ce.gatech.edu/data/GATech_6720/Lab05/01_IDBA_Assembly.tar.gz
tar -xzvf 01_IDBA_Assembly.tar.gz

mkdir -p 02_MaxBin_MAGs
run_MaxBin.pl -contig 01_IDBA_Assembly.tar.gz -reads 00_Reads_QCed/T4AerOil_sbsmpl5.fa -out 02_MaxBin_MAGs/Lab6_MAG

## Step 04: Evaluate the recovered MAGs
Now that we have clustered our assembled contigs into MAGs, we want to learn something about them. In theory, each MAG should represent a single sequence-discrete population (species) living in the environment where the metagenomic sample was collected. In practice, the automated (or even manual) clustering process is filled with noise and uncertainty. Furthermore, we would like to know something about the taxonomic assignments for the bins we’ve recovered. We will use MiGA to evaluate some common genomic metrics and to identify the closest taxonomic assignments of our MAG(s). 

http://microbial-genomes.org/

In [1]:
%%bash
# shortcut
wget http://enve-omics.ce.gatech.edu/data/GATech_6720/Lab05/02_MaxBin_MAGs.tar.gz
tar -xzvf 02_MaxBin_MAGs.tar.gz

--2022-10-04 00:33:48--  http://enve-omics.ce.gatech.edu/data/GATech_6720/Lab05/02_MaxBin_MAGs.tar.gz
Resolving enve-omics.ce.gatech.edu (enve-omics.ce.gatech.edu)... 128.61.130.232
Connecting to enve-omics.ce.gatech.edu (enve-omics.ce.gatech.edu)|128.61.130.232|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1311285 (1.2M) [application/x-gzip]
Saving to: ‘02_MaxBin_MAGs.tar.gz’

     0K .......... .......... .......... .......... ..........  3%  471K 3s
    50K .......... .......... .......... .......... ..........  7% 1.30M 2s
   100K .......... .......... .......... .......... .......... 11% 2.11M 1s
   150K .......... .......... .......... .......... .......... 15% 2.57M 1s
   200K .......... .......... .......... .......... .......... 19% 1.86M 1s
   250K .......... .......... .......... .......... .......... 23% 10.2M 1s
   300K .......... .......... .......... .......... .......... 27% 3.15M 1s
   350K .......... .......... .......... .......... ........

02_MaxBin_MAGs/
02_MaxBin_MAGs/Lab5_MAG.002.fasta
02_MaxBin_MAGs/Lab5_MAG.noclass
02_MaxBin_MAGs/Lab5_MAG.log
02_MaxBin_MAGs/Lab5_MAG.marker_of_each_bin.tar
02_MaxBin_MAGs/Lab5_MAG.abund1
02_MaxBin_MAGs/Lab5_MAG.marker
02_MaxBin_MAGs/Lab5_MAG.001.fasta
02_MaxBin_MAGs/Lab5_MAG.tooshort
02_MaxBin_MAGs/Lab5_MAG.summary


## Step 05: Build recruitment plots with Enveomics RecPlot2
Recruitment plots are used to visualize the distribution of metagenomic reads to a reference genome such as a MAG. Based on this distribution it is possible to infer if a population is heterogeneous or clonal, if there is another closely related population in the metagenome, where the sequence-discrete threshold is, and if any genes or genomic regions from the reference appear to be missing in the metagenome population. We will use the BlastTab.recplot2.R script from the Enveomics collection for this task.

In [ ]:
%%bash

wget http://enve-omics.ce.gatech.edu/data/GATech_6720/Lab05/00_Scripts.tar.gz
tar -xzvf 00_Scripts.tar.gz
mkdir -p 03_RecPlot

wget https://cran.r-project.org/src/contrib/Archive/enveomics.R/enveomics.R_1.5.0.tar.gz
sudo R CMD INSTALL ./enveomics.R_1.5.0.tar.gz

for mag in 02_MaxBin_MAGs/Lab5_MAG*fasta; do
    makeblastdb -dbtype nucl -in $mag    
    blastn -db $mag -query 00_Reads_QCed/T4AerOil_sbsmpl5.fa -out "03_RecPlot/blastoutput_$(basename ${mag}).blast" -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qlen slen'
    python3 00_Scripts/BlastTab_Filter.py -i "03_RecPlot/blastoutput_$(basename ${mag}).blast" -o "03_RecPlot/blastoutput_$(basename ${mag}).filtered.blast"
    BlastTab.catsbj.pl $mag "03_RecPlot/blastoutput_$(basename ${mag}).filtered.blast"
    BlastTab.recplot2.R --prefix "$(basename ${mag})" recplotOutput.Rdata recplotOutput.pdf
    Rscript 00_Scripts/Recplot2_Summary_Stats.R "$(basename ${mag})"recplotOutput.Rdata
done